<a href="https://colab.research.google.com/github/am-bean/face_recognition_bounty/blob/main/face_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# The below dataset below can be generated by running the script `zip_faces.py` with arguments corresponding to the right paths

In [1]:
!unzip -q gdrive/My\ Drive/data/facenoface.zip -d train

unzip:  cannot find or open gdrive/My Drive/data/facenoface.zip, gdrive/My Drive/data/facenoface.zip.zip or gdrive/My Drive/data/facenoface.zip.ZIP.


In [ ]:
import tensorflow as tf
from pathlib import Path

In [ ]:
IMG_SIZE = (64, 64)
BATCH_SIZE = 32
train_dir = Path("train")

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

Found 6454 files belonging to 2 classes.


In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input


In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False
# Let's take a look at the base model architecture
base_model.summary()

image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)

prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)



9406464/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 Conv1 (Conv2D)                 (None, 32, 32, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 32, 32, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 32, 32, 32)   0           ['bn_Conv1[0][0]']               
                

In [ ]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
initial_epochs = 10

history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=train_dataset)

Epoch 1/10
202/202 [==============================] - 29s 126ms/step - loss: 0.7268 - accuracy: 0.6275 - val_loss: 0.4146 - val_accuracy: 0.7615
Epoch 2/10
202/202 [==============================] - 25s 122ms/step - loss: 0.3695 - accuracy: 0.8023 - val_loss: 0.2706 - val_accuracy: 0.8667
Epoch 3/10
202/202 [==============================] - 25s 122ms/step - loss: 0.2650 - accuracy: 0.8703 - val_loss: 0.2187 - val_accuracy: 0.9015
Epoch 4/10
202/202 [==============================] - 25s 120ms/step - loss: 0.2267 - accuracy: 0.8951 - val_loss: 0.1910 - val_accuracy: 0.9185
Epoch 5/10
202/202 [==============================] - 27s 130ms/step - loss: 0.2000 - accuracy: 0.9128 - val_loss: 0.1744 - val_accuracy: 0.9309
Epoch 6/10
202/202 [==============================] - 25s 121ms/step - loss: 0.1844 - accuracy: 0.9275 - val_loss: 0.1628 - val_accuracy: 0.9345
Epoch 7/10
202/202 [==============================] - 25s 123ms/step - loss: 0.1735 - accuracy: 0.9314 - val_loss: 0.1543 - val_ac

In [ ]:
model.save("class_face_v2.h5")